## Load dataset

In [13]:
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
!pip install -q kaggle

In [ ]:
# download api token from: https://www.kaggle.com/settings -> and upload here:
from google.colab import files

files.upload()

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d nguyentuannguyen/optiver-trading-at-the-close
!unzip optiver-trading-at-the-close.zip

In [ ]:
train_dataset = pd.read_csv('/content/optiver-trading-at-the-close/train.csv')
train_dataset.shape

## Local

In [14]:
train_dataset = pd.read_csv('data/optiver-trading-at-the-close/train.csv')
train_dataset.shape

(5237980, 17)

## Feature selection and engineering

In [15]:
df_train = train_dataset.copy()

def drop_missing_values(df):
    # drop the missing values of wap
    df = df.dropna(subset=['wap'])
    return df
# drop the missing values of wap
df_train = drop_missing_values(df_train)
df_train.shape





(5237760, 17)

##### keep only the features selected in "trading_at_the_close_visualisation"
 ['stock_id' 'date_id' 'seconds_in_bucket' 'imbalance_size'
 'reference_price' 'matched_size' 'far_price' 'near_price' 'bid_price'
 'bid_size' 'ask_price' 'ask_size' 'wap' 'row_id' 'date_id_week'
 'date_id_seconds' 'total_volume' 'near_far_ratio' 'near_far_imbalance'
 'ask_mat_ratio' 'bid_mat_ratio' 'ask_ref_ratio'
 'imbalance_continuous_ratio' 'ask_wap_diff' 'all_prices_skew'
 'all_sizes_skew' 'all_prices_kurt' 'all_sizes_kurt'
 'imbalance_buy_sell_flag_cumsum' 'imbalance_buy_sell_flag_0'
 'imbalance_buy_sell_flag_1']


In [16]:


%%time

def calc_feature_single_stock(df):
    # Feature of each stock at different time point
    # date_id
    df["date_id_week"] = df["date_id"] % 5  # Day of the week
    df["date_id_seconds"] = df["seconds_in_bucket"] % 60  # Seconds
    
    
    # ask bid 

    df['mid_price'] = (df['bid_price'] + df['ask_price'])/2
    
    # Volume represents the total quantity of shares that change hands between buyers and sellers.
    df["total_volume"] = df['ask_size'] + df['bid_size']
    
    # near_price far_price
    df['near_far_ratio'] = df['near_price'] / df['far_price']
    df['near_far_imbalance'] = (df['far_price'] - df['near_price']) / (df['far_price'] + df['near_price'])
   
    
    #The ratio between ask/bid and matched sizes provides insights into market liquidity and order flow:
    # A high ask-to-matched ratio suggests that there are more sellers than buyers at a given price level, 
    # potentially indicating resistance to upward price movement.
    # Conversely, a low ratio implies stronger buying interest and potential support for price increases.
    
    df['ask_mat_ratio'] = df['ask_size']/df['matched_size']
    df['bid_mat_ratio'] = df['bid_size']/df['matched_size']
    
    # The ratio between the ask/bid price and the reference price provides insights into market sentiment:
    # - Above 1: If the ask/bid price is higher than the reference price, it suggests bullish sentiment. Sellers are demanding a premium.
    # - Below 1: If the ask/bid price is lower than the reference price, it indicates bearish sentiment. Buyers are getting a discount.
    df['ask_ref_ratio'] = df['ask_price']/df['reference_price']
    
    # imbalance_size matched_size
    df['auction_volume'] = df['imbalance_size'] + df['matched_size']

    # cross term 
    df["imbalance_continuous_ratio"] = df['imbalance_size'] / df["total_volume"]
    # the overall level of buying and selling in financial markets
    df['market_activity'] = df['bid_size'] * df['bid_price'] + df['ask_size'] * df['ask_price']
    # The difference between the ask/bid price and the WAP can indicate market efficiency.
    df['ask_wap_diff'] = df['ask_price'] - df['wap']
    
    # Accumulative features
    df['imbalance_buy_sell_flag_cumsum'] =  df.groupby(['stock_id','date_id'])['imbalance_buy_sell_flag'].cumsum()
    
    # statistical features at different time point
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap"]
    sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]
    for func in [ "skew", "kurt"]:
            df[f"all_prices_{func}"] = df[prices].agg(func, axis=1)
            df[f"all_sizes_{func}"] = df[sizes].agg(func, axis=1)
            

    
    
    return df

df_train = calc_feature_single_stock(df_train)
df_train.tail()


CPU times: total: 1.22 s
Wall time: 5.67 s


,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,...,ask_ref_ratio,auction_volume,imbalance_continuous_ratio,market_activity,ask_wap_diff,imbalance_buy_sell_flag_cumsum,all_prices_skew,all_sizes_skew,all_prices_kurt,all_sizes_kurt
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,...,1.000117,30721084.63,6.931520,352268.485763,0.000106,-54,-0.881646,1.963943,-1.843661,3.871638
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,...,1.000257,9537209.58,1.170884,298717.408464,0.000081,-55,-0.775184,1.996729,-0.152383,3.989047
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,...,1.000094,12725436.10,0.000000,196017.056767,0.000086,-54,2.411565,1.999016,0.000000,3.996681
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,...,1.000000,95774169.89,1.258162,794866.103858,0.000202,28,-1.015717,1.999647,-1.589788,3.998828
5237979,199,480,540,1884285.71,-1,1.002129,24073677.32,1.000859,1.001494,1.002129,...,1.000317,25957963.03,3.424424,551515.933405,0.000173,-35,-1.250444,1.974998,0.696132,3.911662


In [17]:
# make dummy from imbalance_buy_sell_flag and get dummy columns
dummy_cols_list = ['imbalance_buy_sell_flag']
dummy_names = []

for col in dummy_cols_list:
    dummy_df = pd.get_dummies(df_train.loc[:, col], prefix=col, drop_first=True)
    dummy_names += dummy_df.columns.tolist()
    # drop original column
    df_train = df_train.drop(col, axis=1)
    df_train = pd.concat([df_train, dummy_df], axis=1)

In [18]:
features = ['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size'
 ,'reference_price', 'matched_size' ,'far_price', 'near_price' ,'bid_price',
 'bid_size', 'ask_price' ,'ask_size' ,'wap' ,'row_id', 'date_id_week',
 'date_id_seconds', 'total_volume' ,'near_far_ratio', 'near_far_imbalance',
 'ask_mat_ratio' ,'bid_mat_ratio', 'ask_ref_ratio',
 'imbalance_continuous_ratio' ,'ask_wap_diff', 'all_prices_skew',
 'all_sizes_skew' ,'all_prices_kurt', 'all_sizes_kurt',
 'imbalance_buy_sell_flag_cumsum', 'imbalance_buy_sell_flag_0',
 'imbalance_buy_sell_flag_1','target']

In [19]:
df_train = df_train[features]

Split the data

In [20]:
from sklearn.model_selection import train_test_split
train_df = df_train.drop(columns=['target'])
targets = df_train['target']
X_train, X_test, y_train, y_test = train_test_split(train_df, targets, test_size=0.2, shuffle=False)